
# Taiwan IC (HS 8542) Exports — Online Notebook

This notebook can be run **entirely online** (e.g., in your course Lab / Colab / GitHub Codespaces) without installing anything locally.

It will:
1. Load raw CSV & JSON mapping **directly from your GitHub repo (raw URLs)**.
2. Clean & translate market names to English using `country_name_map_full.json`.
3. Compute **Top 12 markets by average exports (2013–2025)**.
4. Produce:
   - A **trend line** chart (Top 12) and save a PNG.
   - An **interactive animated bar chart** (bar chart race style) and save to HTML.


In [ ]:

# ====== USER CONFIG ======
REPO_BASE_URL = (
    "https://raw.githubusercontent.com/a0013333j-png/data-visualization-labs/main/"
    "self-extended-practice/semiconductor-tariff-impact-taiwan/"
)

RAW_CSV_PATH   = "data/raw/taiwan_exports_by_country_2013_2025.csv"
MAP_JSON_PATH  = "data/mappings/country_name_map_full.json"

PNG_TREND      = "taiwan_ic_top12_trend_en.png"
HTML_BARCHART  = "taiwan_ic_top12_barchart.html"

YEAR_MIN, YEAR_MAX = 2013, 2025
TOP_K = 12
EXCLUDE_OTHERS = True


In [ ]:

import io
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import requests

plt.rcParams['figure.dpi'] = 160


In [ ]:

def read_github_raw(url: str, kind: str = "csv"):
    resp = requests.get(url)
    resp.raise_for_status()
    if kind == "csv":
        return pd.read_csv(io.StringIO(resp.text))
    elif kind == "json":
        return json.loads(resp.text)
    else:
        raise ValueError("Unsupported kind")
    
csv_url = REPO_BASE_URL + RAW_CSV_PATH
map_url = REPO_BASE_URL + MAP_JSON_PATH

print("Loading CSV from:", csv_url)
df = read_github_raw(csv_url, kind="csv")

print("Loading mapping JSON from:", map_url)
name_map = read_github_raw(map_url, kind="json")

df.columns = [c.strip() for c in df.columns]
col_year   = next(c for c in df.columns if c.lower()=="year")
col_ctry   = next(c for c in df.columns if c.lower()=="country")
col_value  = next(c for c in df.columns if "export" in c.lower() and "value" in c.lower())

df[col_year] = pd.to_numeric(df[col_year], errors="coerce")
df = df[(df[col_year] >= YEAR_MIN) & (df[col_year] <= YEAR_MAX)].copy()

df["market_en"] = df[col_ctry].map(name_map).fillna(df[col_ctry])
if EXCLUDE_OTHERS:
    df = df[~df["market_en"].str.lower().isin(["other","others","unknown","misc"])].copy()

df[col_value] = pd.to_numeric(df[col_value], errors="coerce").fillna(0)

print("Rows after cleaning:", len(df))
df.head(3)


In [ ]:

avg = (
    df.groupby("market_en")[col_value]
      .mean()
      .sort_values(ascending=False)
)
top_markets = avg.head(TOP_K).index.tolist()
print("Top markets:", top_markets)

df_top = df[df["market_en"].isin(top_markets)].copy()
df_top.head(3)


In [ ]:

pivot = df_top.pivot_table(index=col_year, columns="market_en", values=col_value, aggfunc="sum").fillna(0)

fig, ax = plt.subplots(figsize=(9,5.5))
for m in top_markets:
    ax.plot(pivot.index, pivot[m], label=m, linewidth=2)

ax.set_title("Taiwan IC (HS 8542) Exports — Top 12 Markets Trend ({}–{})".format(YEAR_MIN, YEAR_MAX))
ax.set_xlabel("Year")
ax.set_ylabel("Exports (USD, Millions)")
ax.legend(title="Market", ncol=3, fontsize=9)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(PNG_TREND, bbox_inches="tight")
print("Saved:", PNG_TREND)
plt.show()


In [ ]:

agg = (df_top.groupby([col_year, "market_en"])[col_value].sum().reset_index())

def topk_each_year(frame, k=TOP_K):
    out = []
    for y, g in frame.groupby(col_year):
        g = g.sort_values(col_value, ascending=False).head(k)
        out.append(g)
    return pd.concat(out, ignore_index=True)

agg_top = topk_each_year(agg, TOP_K)

fig_anim = px.bar(
    agg_top, 
    x=col_value, y="market_en", color="market_en",
    orientation="h",
    animation_frame=col_year, animation_group="market_en",
    range_x=[0, agg_top[col_value].max()*1.1],
    title=f"Taiwan IC (HS 8542) Exports — Top {TOP_K} Markets (Animated)",
    labels={col_value: "Exports (USD, Millions)", "market_en": "Market", col_year: "Year"},
    height=600
)

fig_anim.update_layout(showlegend=False, xaxis_title="Exports (USD, Millions)")

fig_anim.write_html(HTML_BARCHART, include_plotlyjs="cdn")
print("Saved:", HTML_BARCHART)

fig_anim.show()



## Outputs
- Trend PNG: `taiwan_ic_top12_trend_en.png`  
- Interactive bar chart (HTML): `taiwan_ic_top12_barchart.html`  

If you're running in a hosted Lab environment, you can usually **download** these files from the file panel on the left, or right-click → Download.
